# Welcome to AI security support system

Have you ever considered the job of a security officer. In some schools, office buildings and even appartements, there is often one individual who is tasked with observing feeds from multiple security cameras. These security guards are capable of making mistakes. Now let us consider the situation of a theft or burglary. It is important for the security officials to be able to easily notified when there is a person on a video feed. That is what we will be solving today. We will be creating a system that is able to detect if there is a person detected on a video feed and alerting the user about it. 

In order to detect the people from the  camera we will be using the 'person-detection-retail-0013' pre-trained models from the OpenVINO model zoo. You can find out more about it here ->https://docs.openvinotoolkit.org/2019_R1/usergroup1.html


In [3]:
import cv2 #importing all required libraries
import numpy as np
from utils.opv import OpvModel  
import matplotlib.pyplot as plt
#import winsound

## Task: Load the pretrained model 

In [5]:
mymodel2 = OpvModel("person-detection-retail-0013",device="CPU", fp="FP16", ncs=0)
#depending on whether you are using Neural Compute Stick you can set the value of ncs to 0 or 1, if not using NCS change device from Myriad to CPU

## Task: Draw Bounding Box around the person

We create a function to draw a bounding box around the person that has been detected.


In [3]:
def DrawBoundingBoxes(predictions, image): # the input will come from the main code
    canvas = image.copy()                             # copy instead of modifying the original image
    conf = 0
    predictions = predictions[0][0]
    confidence = predictions[:,2]
    topresults = predictions[(confidence>conf)]
    (h,w) = canvas.shape[:2]
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h]) # determine box location
        (xmin, ymin, xmax, ymax) = box.astype("int")# assign box location value to xmin, ymin, xmax, ymax

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)  # make a rectangle
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" persons(s) detected", (50,50),# include text
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    return canvas

## Task: The main code

Here is the main code. It turns on our webcam, capture an image upon user request, process that image by calling other user-defined functions, and compare the resulting vector into our vector database. 

For this part of the code we will be using a live video feed.

In [4]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)



while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    predictions = mymodel2.Predict(frame)
    
    cv2.imshow('Press Spacebar to Exit',DrawBoundingBoxes(predictions,frame))
    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()

Seems that we are drawing far too many boxes. However, not every bounding box may be a face. To see which are valid predictions, we will need to compare the probability scores against a threshold that we set.

## Task: Create a threshold.

If there is a person detected on the screen then alert the user through a "beep" sound.

In [5]:
def DrawBoundingBoxes(predictions, image, conf = 0.1): # the predictions and image come from the main code
    canvas = image.copy()                             # copy instead of modifying the original image
    predictions = predictions[0][0]
    confidence = predictions[:,2]
    topresults = predictions[(confidence>conf)]
    (h,w) = canvas.shape[:2]
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h]) # determine box location
        (xmin, ymin, xmax, ymax) = box.astype("int")# assign box location value to xmin, ymin, xmax, ymax

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4) # make a rectangle
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" persons(s) detected", (50,50),# include text
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    if len(topresults) > 0:
        winsound.Beep(3000,20)
    
    return canvas

Let's try again with the modified Confidence interval!

In [6]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)



while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    predictions = mymodel2.Predict(frame)
    
    cv2.imshow('Press Spacebar to Exit',DrawBoundingBoxes(predictions,frame))
    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()


In [19]:
from ipywebrtc import CameraStream, ImageRecorder

In [20]:
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })
camera

CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 640, 'height': 480}})